### 1.

In [36]:
from neo4j import GraphDatabase
import pandas as pd

def get_dungeon_gold(dungeon_name):
    uri = "bolt://localhost:7687"
    driver = GraphDatabase.driver(uri,  auth=("neo4j", "BDII2023"))
    with driver.session() as session:
        query = '''
        MATCH (:Area)-[i:IS_CONNECTED]->(startRoom:Room)
        MATCH (:Area)<-[i2:IS_CONNECTED]-(:Room)
        WHERE i.dungeon_name = $dungeon_name AND i2.dungeon_name = $dungeon_name
        CALL apoc.path.subgraphNodes(startRoom, {
            relationshipFilter: "IS_CONNECTED",
            labelFilter: "Room"
        }) YIELD node
        MATCH (node)-[:CONTAINS]->(l:Loot)
        RETURN sum(l.gold) as gold

        '''
        result = session.run(query, dungeon_name=dungeon_name)
        data = pd.DataFrame([r.values() for r in result], columns=result.keys())
        
    return data

get_dungeon_gold('Fenglass, Sepulcher of the Terrible Emperors')

,gold
0,1227.0


### 2.

In [37]:
def get_mean_monster_lvl(dungeon_name):
    uri = "bolt://localhost:7687"
    driver = GraphDatabase.driver(uri,  auth=("neo4j", "BDII2023"))
    with driver.session() as session:
        query = '''
        MATCH (:Area)-[i:IS_CONNECTED]->(startRoom:Room)
        MATCH (:Area)<-[i2:IS_CONNECTED]-(:Room)
        WHERE i.dungeon_name = $dungeon_name AND i2.dungeon_name = $dungeon_name
        CALL apoc.path.subgraphNodes(startRoom, {
            relationshipFilter: "IS_CONNECTED",
            labelFilter: "Room"
        }) YIELD node
        MATCH (node)-[:CONTAINS]->(m:Monster)
        RETURN avg(m.level) as average_level

        '''
        result = session.run(query, dungeon_name=dungeon_name)
        data = pd.DataFrame([r.values() for r in result], columns=result.keys())
        
    return data

get_mean_monster_lvl('Fenglass, Sepulcher of the Terrible Emperors')

,average_level
0,8.366972


### 3.

In [43]:
def get_mean_relationships_in_dungeon(dungeon_name):
    uri = "bolt://localhost:7687"
    driver = GraphDatabase.driver(uri,  auth=("neo4j", "BDII2023"))
    with driver.session() as session:
        query = '''
        MATCH (:Area)-[i:IS_CONNECTED]->(startRoom:Room)
        MATCH (:Area)<-[i2:IS_CONNECTED]-(:Room)
        WHERE i.dungeon_name = $dungeon_name AND i2.dungeon_name = $dungeon_name
        CALL apoc.path.subgraphNodes(startRoom, {
            relationshipFilter: "IS_CONNECTED",
            labelFilter: "Room"
        }) YIELD node
        MATCH (node)-[r:IS_CONNECTED]-(:Room)
        RETURN node.room_id as id , count(r) as number_of_relationships
        '''
        result = session.run(query, dungeon_name=dungeon_name)
        data = pd.DataFrame([r.values() for r in result], columns=result.keys())
        avg = data['number_of_relationships'].mean()
        # make a groupby to count the number of relationships and get the mean

        
    return avg

get_mean_relationships_in_dungeon('Fenglass, Sepulcher of the Terrible Emperors')

4.31858407079646

### 4.

In [49]:
def get_max_monster_lvl(dungeon_name):
    uri = "bolt://localhost:7687"
    driver = GraphDatabase.driver(uri,  auth=("neo4j", "BDII2023"))
    with driver.session() as session:
        query = '''
        MATCH (:Area)-[i:IS_CONNECTED]->(startRoom:Room)
        MATCH (:Area)<-[i2:IS_CONNECTED]-(:Room)
        WHERE i.dungeon_name = $dungeon_name AND i2.dungeon_name = $dungeon_name
        CALL apoc.path.subgraphNodes(startRoom, {
            relationshipFilter: "IS_CONNECTED",
            labelFilter: "Room"
        }) YIELD node
        MATCH (node)-[:CONTAINS]->(m:Monster)
        RETURN m.name as monster_name, max(m.level) as max_level, node.room_id as room_id, node.room_name as room_name

        '''
        result = session.run(query, dungeon_name=dungeon_name)
        data = pd.DataFrame([r.values() for r in result], columns=result.keys())
        max_level_row = data.loc[data['max_level'].idxmax()]
        
    return max_level_row

get_max_monster_lvl('Fenglass, Sepulcher of the Terrible Emperors')

monster_name             fire giant
max_level                        50
room_id                         289
room_name       magnificent cellar 
Name: 24, dtype: object

### 5.

In [56]:
def get_exp_per_room(dungeon_name):
    uri = "bolt://localhost:7687"
    driver = GraphDatabase.driver(uri,  auth=("neo4j", "BDII2023"))
    with driver.session() as session:
        query = '''
        MATCH (:Area)-[i:IS_CONNECTED]->(startRoom:Room)
        MATCH (:Area)<-[i2:IS_CONNECTED]-(:Room)
        WHERE i.dungeon_name = $dungeon_name AND i2.dungeon_name = $dungeon_name
        CALL apoc.path.subgraphNodes(startRoom, {
            relationshipFilter: "IS_CONNECTED",
            labelFilter: "Room"
        }) YIELD node
        MATCH (node)-[:CONTAINS]->(m:Monster)
        RETURN node.room_id as room_id, node.room_name as room_name, sum(m.exp) as total_exp

        '''
        result = session.run(query, dungeon_name=dungeon_name)
        data = pd.DataFrame([r.values() for r in result], columns=result.keys())
        # sort by total_exp
        data = data.sort_values(by='total_exp', ascending=False)
        data.reset_index(drop=True, inplace=True)
        
    return data

get_exp_per_room('Fenglass, Sepulcher of the Terrible Emperors').head()

,room_id,room_name,total_exp
0,289,magnificent cellar,5000
1,287,chapel,4600
2,321,fancy vault of sumo wrestlers,4600
3,320,jolly washroom,4000
4,477,wine cellar,3600


### 6.

In [59]:
def most_exp_room(dungeon_name):
    room = get_exp_per_room(dungeon_name=dungeon_name).iloc[0][['room_id', 'room_name']]
    return room

most_exp_room('Fenglass, Sepulcher of the Terrible Emperors')

room_id                      289
room_name    magnificent cellar 
Name: 0, dtype: object